In [18]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [19]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [20]:
# Check the number of unique values in each column
attrition_df[attrition_cat].nunique()

#Use the nunique method to check for unique values in the attrition data.

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [21]:
# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df,left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat,1)
attrition_df.head()

# Use merge and drop to replace unencoded categorical variables in the DataFrame.

/var/folders/t1/0t44y88d3sx53f33r21mdfrw0000gn/T/ipykernel_15114/202137446.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  attrition_df = attrition_df.drop(attrition_cat,1)


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [22]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/var/folders/t1/0t44y88d3sx53f33r21mdfrw0000gn/T/ipykernel_15114/3972905584.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values


In [23]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 448       
                                                                 
 dense_4 (Dense)             (None, 5)                 45        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
35/35 [==============================] - 2s 2ms/step - loss: 1907.9011 - accuracy: 0.1733
Epoch 2/100
35/35 [==============================] - 0s 2ms/step - loss: 766.4713 - accuracy: 0.2160
Epoch 3/100
35/35 [==============================] - 0s 2ms/step - loss: 57.3307 - accuracy: 0.6652
Epoch 4/100
35/35 [==============================] - 0s 3ms/step - loss: 13.4677 - accuracy: 0.8203
Epoch 5/100
35/35 [==============================] - 0s 3ms/step - loss: 9.1163 - accuracy: 0.8158
Epoch 6/100
35/35 [==============================] - 0s 3ms/step - loss: 6.3434 - accuracy: 0.8176
Epoch 7/100
35/35 [==============================] - 0s 2ms/step - loss: 4.3005 - accuracy: 0.8212
Epoch 8/100
35/35 [==============================] - 0s 2ms/step - loss: 2.8691 - accuracy: 0.8221
Epoch 9/100
35/35 [==============================] - 0s 3ms/step - loss: 1.9479 - accuracy: 0.8230
Epoch 10/100
35/35 [==============================] - 0s 3ms/step - loss: 1.3202 - accuracy: 0.8258
Ep

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 1s - loss: 0.3936 - accuracy: 0.8723 - 517ms/epoch - 43ms/step
Loss: 0.3936085104942322, Accuracy: 0.8722826242446899


In [26]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
27/35 [======================>.......] - ETA: 0s - loss: 0.5967 - accuracy: 0.7951 
Epoch 1: saving model to checkpoints/weights.01.hdf5
35/35 [==============================] - 1s 5ms/step - loss: 0.5901 - accuracy: 0.7995
Epoch 2/100
16/35 [============>.................] - ETA: 0s - loss: 0.5387 - accuracy: 0.8418
Epoch 2: saving model to checkpoints/weights.02.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.5303 - accuracy: 0.8258
Epoch 3/100
32/35 [==========================>...] - ETA: 0s - loss: 0.4850 - accuracy: 0.8281
Epoch 3: saving model to checkpoints/weights.03.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 0.4848 - accuracy: 0.8276
Epoch 4/100
34/35 [============================>.] - ETA: 0s - loss: 0.4527 - accuracy: 0.8281
Epoch 4: saving model to checkpoints/weights.04.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 0.4525 - accuracy: 0.8276
Epoch 5/100
35/35 [==============================] - ETA: 0

35/35 [==============================] - 0s 3ms/step - loss: 0.2893 - accuracy: 0.8648
Epoch 36/100
32/35 [==========================>...] - ETA: 0s - loss: 0.2903 - accuracy: 0.8691
Epoch 36: saving model to checkpoints/weights.36.hdf5
35/35 [==============================] - 0s 5ms/step - loss: 0.2869 - accuracy: 0.8702
Epoch 37/100
23/35 [==================>...........] - ETA: 0s - loss: 0.2901 - accuracy: 0.8682
Epoch 37: saving model to checkpoints/weights.37.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.2846 - accuracy: 0.8675
Epoch 38/100
27/35 [======================>.......] - ETA: 0s - loss: 0.2777 - accuracy: 0.8715
Epoch 38: saving model to checkpoints/weights.38.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.2828 - accuracy: 0.8711
Epoch 39/100
23/35 [==================>...........] - ETA: 0s - loss: 0.2903 - accuracy: 0.8696
Epoch 39: saving model to checkpoints/weights.39.hdf5
35/35 [==============================] - 0s 3ms/step

34/35 [============================>.] - ETA: 0s - loss: 0.2148 - accuracy: 0.9182
Epoch 70: saving model to checkpoints/weights.70.hdf5
35/35 [==============================] - 0s 6ms/step - loss: 0.2195 - accuracy: 0.9165
Epoch 71/100
22/35 [=================>............] - ETA: 0s - loss: 0.2223 - accuracy: 0.9219
Epoch 71: saving model to checkpoints/weights.71.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.2178 - accuracy: 0.9174
Epoch 72/100
23/35 [==================>...........] - ETA: 0s - loss: 0.2101 - accuracy: 0.9239
Epoch 72: saving model to checkpoints/weights.72.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.2158 - accuracy: 0.9192
Epoch 73/100
22/35 [=================>............] - ETA: 0s - loss: 0.2164 - accuracy: 0.9247
Epoch 73: saving model to checkpoints/weights.73.hdf5
35/35 [==============================] - 0s 3ms/step - loss: 0.2145 - accuracy: 0.9192
Epoch 74/100
20/35 [================>.............] - ETA: 0s - l

In [28]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3768 - accuracy: 0.8668 - 201ms/epoch - 17ms/step
Loss: 0.37675708532333374, Accuracy: 0.866847813129425


In [29]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [30]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [31]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.3768 - accuracy: 0.8668 - 208ms/epoch - 17ms/step
Loss: 0.37675708532333374, Accuracy: 0.866847813129425
